<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpy

Criando os processos de uma entidade:

In [12]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env):
  while True:
    print('Escavadeira trabalhando em %f' %(env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Começou manutenção da escavadeira em %f' %(env.now))
    repair = np.random.exponential(1)
    yield env.timeout(repair)

env = simpy.Environment()
env.process(escavadeira(env))


env.run(until=50)

Escavadeira trabalhando em 0.000000
Escavadeira trabalhando em 0.000000
Começou manutenção da escavadeira em 5.571122
Escavadeira trabalhando em 6.494345
Começou manutenção da escavadeira em 8.791515
Escavadeira trabalhando em 9.342564
Começou manutenção da escavadeira em 12.004753
Escavadeira trabalhando em 12.580272
Começou manutenção da escavadeira em 16.609679
Escavadeira trabalhando em 19.924591
Começou manutenção da escavadeira em 23.309806
Escavadeira trabalhando em 24.878702
Começou manutenção da escavadeira em 28.144943
Escavadeira trabalhando em 28.984376
Começou manutenção da escavadeira em 30.147421
Escavadeira trabalhando em 30.221106
Começou manutenção da escavadeira em 30.859233
Escavadeira trabalhando em 30.879659
Começou manutenção da escavadeira em 43.392073
Escavadeira trabalhando em 44.897857
Começou manutenção da escavadeira em 47.172155


Trabalhando com mais de uma entidade do mesmo tipo

In [14]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env, nome):
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
    repair = np.random.exponential(1)
    yield env.timeout(repair)

env = simpy.Environment()
env.process(escavadeira(env, '%d' % 1))


env.run(until=50)

Escavadeira 1 trabalhando em 0.000000
Começou manutenção da escavadeira 1 em 5.571122
Escavadeira 1 trabalhando em 6.827052
Começou manutenção da escavadeira 1 em 13.289614
Escavadeira 1 trabalhando em 14.076815
Começou manutenção da escavadeira 1 em 17.934155
Escavadeira 1 trabalhando em 18.972314
Começou manutenção da escavadeira 1 em 23.000949
Escavadeira 1 trabalhando em 25.224473
Começou manutenção da escavadeira 1 em 48.428858
Escavadeira 1 trabalhando em 48.912460


Trabalhando com recursos com capacidade limitada

In [ ]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env, nome, eqp):
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Escavadeira %s quebrou em %f' %(nome, env.now))
    with eqp.request() as req:
      yield req
      print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
      repair = np.random.exponential(1)
      yield env.timeout(repair)

env = simpy.Environment()
eqp = simpy.Resource(env, capacity=1)
for i in range(7):
  env.process(escavadeira(env, '%d' % i, eqp))



env.run(until=50)


Pegando o tempo de ocupação do recurso:

In [25]:
import simpy
import numpy as np

np.random.seed(0)

ocp = 0
def escavadeira(env, nome, eqp):
  global ocp
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Escavadeira %s quebrou em %f' %(nome, env.now))


    with eqp.request() as req:
      yield req
      print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
      repair = np.random.exponential(1)
      ocp += repair
      yield env.timeout(repair)

env = simpy.Environment()
eqp = simpy.Resource(env, capacity=1)
for i in range(7):
  env.process(escavadeira(env, '%d' % i, eqp))



env.run(until=10)
print(f"Tempo total que o recurso 'eqp' ficou ocupado: {ocp}")


Escavadeira 0 trabalhando em 0.000000
Escavadeira 1 trabalhando em 0.000000
Escavadeira 2 trabalhando em 0.000000
Escavadeira 3 trabalhando em 0.000000
Escavadeira 4 trabalhando em 0.000000
Escavadeira 5 trabalhando em 0.000000
Escavadeira 6 trabalhando em 0.000000
Escavadeira 4 quebrou em 3.857339
Começou manutenção da escavadeira 4 em 3.857339
Escavadeira 6 quebrou em 4.028634
Escavadeira 3 quebrou em 5.510408
Escavadeira 0 quebrou em 5.571122
Escavadeira 4 trabalhando em 6.080864
Começou manutenção da escavadeira 6 em 6.080864
Escavadeira 2 quebrou em 6.462562
Escavadeira 6 trabalhando em 6.564466
Começou manutenção da escavadeira 3 em 6.564466
Escavadeira 5 quebrou em 7.267115
Escavadeira 3 trabalhando em 7.317140
Começou manutenção da escavadeira 0 em 7.317140
Escavadeira 1 quebrou em 8.791515
Escavadeira 0 trabalhando em 9.915394
Começou manutenção da escavadeira 2 em 9.915394
Tempo total que o recurso 'eqp' ficou ocupado: 6.14921580651594
